In [ ]:
%env GOOGLE_APPLICATION_CREDENTIALS = /Users/blevine/goonalytics/resources/gcloud-cred.json

In [10]:
from goonalytics.io import gcloudio as gc
from goonalytics.io.gcloudio import BigQueryer
from goonalytics.settings import GCLOUD_TABLE_NAME
from goonalytics.base import User

tbl = GCLOUD_TABLE_NAME

In [11]:
def rndusr(min_val=0):
    query = ''' select user_id, user_name, min(rand()) as rand from %s group by user_id, user_name having count(user_id) > %d order by rand''' % (GCLOUD_TABLE_NAME,min_val)
    bq = gc.BigQueryer()
    res = bq.sql_inject(query).rows[0]
    return User(user_id=res[0], user_name=res[1])

usr = rndusr(50)
print(usr)

DEBUG:goonalytics.io.gcloudio:Executing synchronous query:  select user_id, user_name, min(rand()) as rand from [empyrean-bridge-150804:forums.f44] group by user_id, user_name having count(user_id) > 50 order by rand


User(user_id=197859, user_name='Junpei Hyde')


In [12]:
def posts_for_user(user: User) -> str:
    query = ''' select post_text from %s where user_id=%d ''' % (GCLOUD_TABLE_NAME, user.user_id)
    bq = BigQueryer()
    qres = bq.sql_inject(query)
    from functools import reduce
    posts = [row[0] for row in qres.rows]
    return reduce(lambda x,y: x + ' ' + y, posts)

p = posts_for_user(usr)
print(len(p))

DEBUG:goonalytics.io.gcloudio:Executing synchronous query:  select post_text from [empyrean-bridge-150804:forums.f44] where user_id=197859 


326408


In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [14]:
cv = TfidfVectorizer(input='content', stop_words='english', analyzer='word', norm='l2')
cv.fit(p)

ValueError: Iterable over raw text documents expected, string object received.